# Model VECM (package [`statsmodels`](https://www.statsmodels.org/stable/index.html))

In [ ]:
import numpy as np
import pandas as pd

from statsmodels.tsa.api import VECM
from statsmodels.tsa.vector_ar.vecm import coint_johansen, select_coint_rank, select_order

import pandas_datareader.data as web

# настройки визуализации
import matplotlib.pyplot as plt

# Не показывать Warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)
# Не показывать ValueWarning, ConvergenceWarning из statsmodels
from statsmodels.tools.sm_exceptions import ValueWarning, ConvergenceWarning
warnings.simplefilter('ignore', category=ValueWarning)
warnings.simplefilter('ignore', category=ConvergenceWarning)

In [ ]:
y = web.DataReader(name=['TB3MS', 'GS10', 'AAA', 'BAA'], data_source='fred', start='1995-01-01')

y.plot(subplots=True)
plt.show()

## Cointegration test

We call [coint_johansen(endog, det_order, k_ar_diff)](https://www.statsmodels.org/stable/generated/statsmodels.tsa.vector_ar.vecm.coint_johansen.html#statsmodels.tsa.vector_ar.vecm.coint_johansen)

where 
* `det_order`
	* -1 - no deterministic terms
	* 0 - constant term
	* 1 - linear trend
* `k_ar_diff`: Number of lagged differences in the model

In [ ]:
# Johansen test
coint_res = coint_johansen(y, det_order=0, k_ar_diff=2)

In [ ]:
# trace-stat & critical
coint_res.trace_stat, coint_res.trace_stat_crit_vals

In [ ]:
# eigenvalue-stat & critical
coint_res.max_eig_stat, coint_res.max_eig_stat_crit_vals

We call [`select_coint_rank(endog, det_order, k_ar_diff, method='trace', signif=0.05)`](https://www.statsmodels.org/stable/generated/statsmodels.tsa.vector_ar.vecm.select_coint_rank.html#statsmodels.tsa.vector_ar.vecm.select_coint_rank)

where 
* `det_order`
	* -1 - no deterministic terms
	* 0 - constant term
	* 1 - linear trend
* `k_ar_diff` - Number of lagged differences in the model
* `method` - test statistics: `trace` or `maxeig`
* `signif` is significant level: 0.1, 0.05, 0.01

In [ ]:
# select cointegration rank
select_coint_rank(y, det_order=0, k_ar_diff=2, method='trace', signif=0.05).summary()

## Fitting VECM

For automatic lag selection we call [`select_order(data, maxlags, deterministic='n')`](https://www.statsmodels.org/stable/generated/statsmodels.tsa.vector_ar.vecm.select_order.html#statsmodels.tsa.vector_ar.vecm.select_order)

where `deterministic` 
* "n" - no deterministic terms
* "co" - constant outside the cointegration relation
* "ci" - constant within the cointegration relation
* "lo" - linear trend outside the cointegration relation
* "li" - linear trend within the cointegration relation

Combinations of these are possible (e.g. "cili" or "colo" for linear trend with intercept)

In [ ]:
# number or lags selection for VECM
select_order(y, maxlags=2, deterministic='c').summary()

For the fitting of the model of given oder we call [VECM](https://www.statsmodels.org/stable/generated/statsmodels.tsa.vector_ar.vecm.VECM.html#statsmodels.tsa.vector_ar.vecm.VECM)

здесь `deterministic` 
* "n" - no deterministic terms
* "co" - constant outside the cointegration relation
* "ci" - constant within the cointegration relation
* "lo" - linear trend outside the cointegration relation
* "li" - linear trend within the cointegration relation

Combinations of these are possible (e.g. "cili" or "colo" for linear trend with intercept)

In [ ]:
mod = VECM(y, k_ar_diff=1, coint_rank=1, deterministic='co', missing='drop')

res = mod.fit()

res.summary(alpha=0.05)

## Model diagnostic

In [ ]:
# serial correlation test
res.test_whiteness(nlags=10, signif=0.05, adjusted=True).summary()

## Forecasting

In [ ]:
# we transform y into numpy array
res.predict(steps=10)

Visualization

In [ ]:
fig = res.plot_forecast(steps=20, alpha=0.05, plot_conf_int=True, n_last_obs=50)

# изменим положение легенды для каждого подграфика
# for i in range(y.shape[1]):
# 	fig.axes[i].legend(loc=3)

plt.show()

## IRF

In [ ]:
res.irf(periods=10).plot(signif=0.05)
plt.show()

## Causality test

In [ ]:
res.test_granger_causality(caused=['AAA', 'BAA'], causing=['TB3MS', 'GS10'], signif=0.01).summary()

In [ ]:
res.test_granger_causality(caused= ['TB3MS', 'GS10'], causing=['AAA', 'BAA'], signif=0.01).summary()